In [1]:
from gurobipy import Model, quicksum, GRB

In [2]:
Plnt = {'plnt'}
DC = set(range(1,3))
Cust = DC.copy()
TRoute = {(i,j) for i in Plnt for j in DC}
DRoute = {(j,k) for j in DC for k in Cust}
Route = TRoute | DRoute
CT = {1,2}

T = 2
Period = list(range(1, T+1))

In [3]:
tp_cost = {(i,j):1 for i,j in TRoute}
del_cost = {(j, k):2 for j,k in DRoute}
demand = {(k,t):10 for k in Cust for t in Period}
dc_run_cost = {(j,ct):1 * ct for j in DC for ct in CT}
dc_inv_cost = {j:0 for j in DC}
dc_ub = {ct:40 * ct for ct in CT}
cust_inv_cost = {k:30 for k in Cust}

In [4]:
dc_ub

{1: 40, 2: 80}

In [5]:
model = Model()

Academic license - for non-commercial use only


In [6]:
x, y, dc_inv, dc_plus_inv, cust_inv, cust_plus_inv = {}, {}, {}, {}, {}, {}

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for j in DC:
    for ct in CT:
        for t in Period:
            y[j,ct,t] = model.addVar(vtype="B", name=f'y[{j},{ct},{t}]')
    
for j in DC:
    dc_plus_inv[j] = model.addVar(vtype="C", name=f'dc_plus_inv[{j}]')
    dc_inv[j,0] = model.addVar(vtype="C", ub=0, name=f'dc_inv[{j},{t}]')  
    for t in Period:
        dc_inv[j,t] = model.addVar(vtype="C", name=f'dc_inv[{j},{t}]')    

for k in Cust:
    cust_plus_inv[k] = model.addVar(vtype="C", name=f'cust_plus_inv[{k}]')
    cust_inv[k,0] = model.addVar(vtype="C", ub=0, name=f'cust_inv[{k},{t}]')  
    for t in Period:
        cust_inv[k,t] = model.addVar(vtype="C", name=f'cust_inv[{k},{t}]')

model.update()

In [7]:
Cust_Demand_Cons, DC_Running_Cons, DC_Flow_Cons, Center_Type_Cons, DC_UB_Cons = {}, {}, {}, {}, {}

for k in Cust:
    for t in Period:
        Cust_Demand_Cons[k,t] = model.addConstr(
            quicksum(x[j,k,t] for j in DC)
            + cust_inv[k,t-1]
            ==
            demand[k,t]
            + cust_inv[k,t]
        )

for j,k in DRoute:
    for t in Period:
        DC_Running_Cons[j,k,t] = model.addConstr(
            x[j,k,t]
            <=
            quicksum(demand[k,t_] for t_ in Period) * quicksum(y[j,ct,t] for ct in CT) 
        )

for j in DC:
    for t in Period:
        DC_Flow_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in Plnt)
            + dc_inv[j,t-1]
            ==
            quicksum(x[j,k,t] for k in Cust)
            + dc_inv[j,t]
        )

for j in DC:
    for t in Period:
        Center_Type_Cons[j,t] = model.addConstr(
            quicksum(y[j,ct,t] for ct in CT)
            <=
            1
        )

for j in DC:
    for t in Period:
        DC_UB_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in Plnt) + dc_inv[j,t-1]
            <=
            quicksum(dc_ub[ct] * y[j,ct,t] for ct in CT)
        )

model.addConstr(x["plnt",1,2]==0)
model.addConstr(x["plnt",2,2]==0)
        
model.update()

In [8]:
model.setObjective(
    quicksum(tp_cost[i,j] * x[i,j,t] for i,j in TRoute for t in Period) +
    quicksum(del_cost[j,k] * x[j,k,t] for j,k in DRoute for t in Period) +
    quicksum(dc_run_cost[j,ct] * y[j,ct,t] for j in DC for ct in CT for t in Period) +
    quicksum(dc_inv_cost[j] * dc_inv[j,t] for j in DC for t in Period) +
    quicksum(dc_inv_cost[j] * dc_plus_inv[j] for j in DC) + 
    quicksum(cust_inv_cost[k] * cust_inv[k,t] for k in Cust for t in Period) +
    quicksum(cust_inv_cost[k] * cust_plus_inv[k] for k in Cust)
    ,GRB.MINIMIZE
)
model.update()

In [9]:
model.optimize()

Optimize a model with 26 rows, 36 columns and 86 nonzeros
Variable types: 28 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 16 rows and 25 columns
Presolve time: 0.00s
Presolved: 10 rows, 11 columns, 28 nonzeros
Variable types: 5 continuous, 6 integer (6 binary)

Root relaxation: objective 1.220000e+02, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  122.00000    0    2          -  122.00000      -     -    0s
H    0     0                     123.0000000  122.00000  0.81%     -    0s
*    0     0               0     122.0000000  122.00000  0.00%     -    0s

Cutting planes:
  Implied bound: 2

Explored 1 nodes (7 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available 

In [10]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

x[1,2,1] 10.0
x[1,2,2] 10.0
x[plnt,1,1] 40.0
x[1,1,1] 10.0
x[1,1,2] 10.0
y[1,1,1] 1.0
y[1,1,2] 1.0
dc_inv[1,1] 20.0
